### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

### Requirements
* Must have an Azure Subscription
* Must [register an App](https://ms.portal.azure.com/#blade/Microsoft_AAD_RegisteredApps/ApplicationsListBlade)
    * Permissions: Will need the Contributor role at the scope where the resources will be created, which is often the subscription level
* Azure Key Vault
    * [Set up](https://learn.microsoft.com/azure/key-vault/secrets/quick-create-portal) within the Azure Subscription
    * Save secrets for the Tenant ID, Client ID (Application ID), Client Secret
    * Permissions: Ensure the user who will be executing the notebooks has “Key Vault Secrets User”
* Fabric Permissions
    * User should be a tenant admin. This ensures they have the necessary authority to execute and manage the notebooks without encountering permission issues.

### Result
* F skus are created for each (specified) capacity
    * Within the same region as the P SKU
    * Equivalent SKU size as the P SKU
    * Same admins as listed on the P SKU
    * All workspaces are migrated to the corresponding new capacity
    * Capacity settings from the P SKU are transferred to the F SKU
        * Capacity settings
        * Notification settings
        * Access settings
        * Disaster recovery settings
        * Spark settings
        * Delegated tenant settings
* The names of the newly created F SKU capacities will be an alphanumeric lowercase version of the P SKU capacity name, suffixed with 'fsku'. As an example: "My capacity_3!" -> "mycapacity3fsku".

### Import the library and set the initial parameters

In [ ]:
import sempy_labs as labs
import sempy_labs.admin as admin

azure_subscription_id = '' # Enter your Azure subscription ID
key_vault_uri = '' # Enter your Azure Key Vault URI
key_vault_tenant_id = '' # Enter the name of the Azure Key Vault secret which stores your Tenant ID
key_vault_client_id = '' # Enter the name of the Azure Key Vault secret which stores your Client ID (Application ID)
key_vault_client_secret = '' # Enter the name of the Azure Key Vault secret which stores your Client Secret
resource_group = '' # Enter the name of the resource group (to be used to create the new F skus)

### Migrate a single P SKU -> F SKU
Set the 'capacities' parameter to the single P SKU.

In [ ]:
labs.migrate_capacities(
    azure_subscription_id = azure_subscription_id,
    key_vault_uri = key_vault_uri,
    key_vault_tenant_id = key_vault_tenant_id,
    key_vault_client_id = key_vault_client_id,
    key_vault_client_secret = key_vault_client_secret,
    resource_group = resource_group,
    capacities = 'CapacityA',
    p_sku_only = True,
)

### Migrate a list of P SKUs to F SKUs
Set the 'capacities' parameter to a list of P SKUs.

In [ ]:
labs.migrate_capacities(
    azure_subscription_id = azure_subscription_id,
    key_vault_uri = key_vault_uri,
    key_vault_tenant_id = key_vault_tenant_id,
    key_vault_client_id = key_vault_client_id,
    key_vault_client_secret = key_vault_client_secret,
    resource_group = resource_group,
    capacities = ['CapacityA', 'CapacityB', 'CapacityC'],
    p_sku_only = True,
)

### Migrate all P SKUs to F SKUs
Set the 'capacities' parameter to None.

In [ ]:
labs.migrate_capacities(
    azure_subscription_id = azure_subscription_id,
    key_vault_uri = key_vault_uri,
    key_vault_tenant_id = key_vault_tenant_id,
    key_vault_client_id = key_vault_client_id,
    key_vault_client_secret = key_vault_client_secret,
    resource_group = resource_group,
    capacities = None,
    p_sku_only = True,
)

### Migrate a list of P SKUs to F SKUs; associate each capacity with a specific resource group
This process ensures that each F SKU is created within the resource group specified in the resource_group_mapping dictionary.

In [ ]:
resource_group_mapping = {
    "CapacityA": "ResourceGroupA",
    "CapacityB": "ResourceGroupA",
    "CapacityC": "ResourceGroupB",
}

labs.migrate_capacities(
    azure_subscription_id = azure_subscription_id,
    key_vault_uri = key_vault_uri,
    key_vault_tenant_id = key_vault_tenant_id,
    key_vault_client_id = key_vault_client_id,
    key_vault_client_secret = key_vault_client_secret,
    resource_group = resource_group_mapping,
    capacities = ['CapacityA', 'CapacityB', 'CapacityC'],
    p_sku_only = True,
)

### Migrate a single source capacity to a target capacity (target capacity already created)

In [ ]:
source_capacity = '' # Enter the Source capacity name
target_capacity = '' # Enter the Target capacity name (already exists) 

admin.assign_workspaces_to_capacity(
    source_capacity=source_capacity,
    target_capacity=target_capacity,
    workspace=None,
)

# Optionally migrate settings
"""
labs.migrate_capacity_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
labs.migrate_access_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
labs.migrate_delegated_tenant_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
labs.migrate_disaster_recovery_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
labs.migrate_notification_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
labs.migrate_spark_settings(
    source_capacity=source_capacity, 
    target_capacity=target_capacity
)
"""

### Migrate a list of source capacities to target capacities (target capacity already created)

In [ ]:
capacity_mapping = {
    "capacitya": "capacityanew", # Format is "Source SKU": "Target SKU"
    "capacityb": "capacitybnew",
    "capacityc": "capacitycnew",
}

capacities = list(capacity_mapping.keys())

for capacity in capacities:
    admin.assign_workspaces_to_capacity(
        source_capacity=capacity,
        target_capacity=capacity_mapping.get(capacity),
        workspace=None,
    )
    # Optionally migrate settings
    """
    labs.migrate_capacity_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    labs.migrate_access_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    labs.migrate_delegated_tenant_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    labs.migrate_disaster_recovery_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    labs.migrate_notification_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    labs.migrate_spark_settings(
        source_capacity=source_capacity, 
        target_capacity=target_capacity
    )
    """


### Migrate a Fabric Trial Capacity (FT1) to a Fabric Capacity (F SKU)

In [ ]:
labs.migrate_fabric_trial_capacity(
    azure_subscription_id = azure_subscription_id,
    key_vault_uri = key_vault_uri,
    key_vault_tenant_id = key_vault_tenant_id,
    key_vault_client_id = key_vault_client_id,
    key_vault_client_secret = key_vault_client_secret,
    resource_group = resource_group,
    source_capacity = '', # The name of the Trial capacity.
    target_capacity = '', # The name of Fabric capacity (if it does not exist it will be created).
    target_capacity_sku = "F64", # Defaults to F64 but can specify the required SKU.
    target_capacity_admin_members = None, # Setting this to None will use the same admin members as the Trial capacity.
)